## Numerical implementation of time averaged 3DVAR on the 2D negative, inverse Laplacian.

A canonical example of a compact linear operator is the negative, inverse Laplacian. We test our algorithm on the 2D Poisson problem. We wish to recover $u\in H = L^2([0,1]^2)$ with periodic boundary conditions given $f$. We know that
$$
-\Delta u = f,
$$
We require that $f\in H$ with $\int f(x,y)dxdy =0$, this is a solvability condition.

We take a Fourier approach.

The real valued, mean zero eigenfunctions of the problem are of the form
$$
\exp i\left(\frac{2\pi}{L_x}k_x x + \frac{2\pi}{L_y}k_y y   \right)
$$
for wave numbers $k_x,k_y=1,2,\dots$.

We obtain a solution by expanding $u$ in terms of its Fourier series using a FFT.
$$
u(x,y) = \sum_{(k_x,k_y)\neq (0,0)}{u}_{k_x,k_y} \exp i\left(\frac{2\pi}{L_x}k_x x + \frac{2\pi}{L_y}k_y y   \right)
$$
$f$ can be analogously expanded. We take $N_x, N_y\in \mathbb{N}^+$ even and truncate $k_x = -\frac{N_x}{2} + 1,\dots, \frac{N_x}{2}$, $k_x = -\frac{N_y}{2} + 1,\dots, \frac{N_y}{2}$. This induces the mesh produced by $\Delta x = \frac{L_x}{N_x}$ and $\Delta y = \frac{L_y}{N_y}$.
By mode matching
$$
{f}_{k_x,k_y} \exp i\left(\frac{2\pi}{L_x}k_x x + \frac{2\pi}{L_y}k_y y   \right),
$$
$$
 = -\Delta u,
$$
$$
 = -\Delta {u}_{k_x,k_y} \exp i\left(\frac{2\pi}{L_x}k_x x + \frac{2\pi}{L_y}k_y y   \right),
 $$
 $$
 =\left[\left(\frac{2\pi}{L_x}k_x\right)^2 + \left(\frac{2\pi}{L_y}k_y \right)^2\right]{u}_{k_x,k_y} \exp i\left(\frac{2\pi}{L_x}k_x x + \frac{2\pi}{L_y}k_y y   \right),
 $$
 $$
{f}_{k_x,k_y}= \left[\left(\frac{2\pi}{L_x}k_x\right)^2 + \left(\frac{2\pi}{L_y}k_y \right)^2\right]{u}_{k_x,k_y}.
$$

The 3DVAR estimator can be written as
$$
u_j = (A^*A+\alpha\Sigma^{-1})^{-1}(A^*f_j + \alpha \Sigma^{-1}u_{j-1})
$$
where for us, 
$$
A=A^* = \mathcal{F}^{-1} \circ \left[\left(\frac{2\pi}{L_x}k_x\right)^2 + \left(\frac{2\pi}{L_y}k_y \right)^2\right]^{-1}_{-\frac{N_x}{2}\leq k_x\leq \frac{N_x}{2},-\frac{N_y}{2}\leq k_y\leq \frac{N_y}{2}}\bullet\circ \mathcal{F}.
$$

Here, $\mathcal{F}$ is the Fourier transform. For simplicity, we use $\Sigma=A^2$. It is well known, (Weyl's theorem, see \cite{folland} for a reference), that $\Delta^{-1}$ has eigenvalues $\lambda_k\asymp \frac{1}{k}$ in dimension 2.  By Lidskii's theorem $\Sigma$ is trace class with eigenvalues $\asymp\frac{1}{k^2}$.


In [ ]:
using Plots; pyplot();
using FFTW;
using LinearAlgebra;
using Random;
Random.seed!(100);

In [ ]:
#  domain [0,Lx)\times [0,Ly)
Lx = 1.0; 
Ly = 2.0;

# number of resolved points in each direction
Nx = 32;
Ny = 64;

# consruct mesh, excluding last point
Δx = Lx/Nx;
Δy = Ly/Ny;

# Regular mesh
x = LinRange(0.0, Lx, Nx+1)[1:end-1];
y = LinRange(0.0, Ly, Ny+1)[1:end-1];

# construct wave numbers in correct layout
kx = [0:Int(Nx/2); -Int(Nx/2)+1:1:-1]; 
ky = [0:Int(Ny/2); -Int(Ny/2)+1:1:-1];


In [ ]:
#Building functions for the forcing term and building exact solution analytically.
kkx = 1;
kky = 2;

f = (x,y) -> [sin(2*π/Lx *kkx * xx)*sin(2*π/Ly *kky * yy) for yy in y, xx in x];
f_noisy = (x,y,γ) -> [sin(2*π/Lx *kkx * xx)*sin(2*π/Ly *kky * yy) + γ^2 * randn(1)[1] for yy in y, xx in x];
u_exact = 1/( (2*π/Lx*kkx)^2 + (2*π/Ly *kky )^2 )* f(x,y);

In [ ]:
function ThreeDVAR_update(u_oldvals,fvals,α,kx,ky)
    #applies the 3DVAR update and returns a new state u_new
    #u_oldvals and fvals are arrays of complex numbers corresponding to the current state and a noisy sample f of the forcing term.
    #α is the regularization parameter and kx,ky are wavenumbers
    
    #appling the fast fourier transform
    fhat = fft(fvals);
    u_oldhat = fft(u_oldvals);
    u_newhat = zeros(ComplexF64,(Ny,Nx));
    
    #applying the 3DVAR update in fourier space
    for j=1:Ny,i=1:Nx
        Ak = ( (2*π/Lx * kx[i])^2 + (2*π/Ly * ky[j])^2 );
        Sigmainv = Ak^(-2)
        if(i+j>2) 
           u_newhat[j,i]= (Ak.*fhat[j,i] + α.*Sigmainv.*u_oldhat[j,i]) ./ (Ak.^2 + α*Sigmainv);
        end
    end
    
    #applying the inverse fast fourier transform
    u_new = ifft(u_newhat);
    return u_new
end


In [ ]:
function run_simulation(num_samples, α, γ)
    #runs both 3DVAR and time averaged 3DVAR on the 2D Poisson problem,
    #returns solutions for the classic and time averaged problems and time series 2-norm errors.
    #num_samples is the number of samples to run, α is the regularization parameter and γ is the noise level.
    u_3DVAR = zeros(ComplexF64,(Ny,Nx));
    u_TA3DVAR = zeros(ComplexF64,(Ny,Nx));

    error_3DVAR=zeros(num_samples);
    error_TA3DVAR=zeros(num_samples);

    for j = 1:num_samples
        fvals = f_noisy(x,y,γ);
        u_3DVAR = ThreeDVAR_update(u_3DVAR,fvals,α,kx,ky); #applying update
        u_TA3DVAR = (j/(j+1))*u_TA3DVAR + (1/(j+1))*u_3DVAR; #updating time average
        
        #calculating errors
        error_3DVAR[j]=norm(u_3DVAR .-u_exact)*sqrt(Δx*Δy);
        error_TA3DVAR[j]=norm(u_TA3DVAR .-u_exact)*sqrt(Δx*Δy);
    end 
    return u_3DVAR, u_TA3DVAR, error_3DVAR, error_TA3DVAR
end


In [ ]:
#Running simulation
num_samples = 100; α = 1; γ = 1;
u_3DVAR, u_TA3DVAR, error_3DVAR, error_TA3DVAR = run_simulation(num_samples, α, γ);

In [ ]:
#Plotting errors
plot(1:num_samples, error_3DVAR, label="3DVAR Error")
plot!(1:num_samples, error_TA3DVAR, label="Time Averaged 3DVAR Error")
plot!(1:num_samples, (ones(num_samples)./((1:num_samples).^(1/2))).*error_TA3DVAR[1], label="theoretical error")
title!("2-norm Error")
xlabel!("Iterations")
ylabel!("Error")
savefig("err")

In [ ]:
#running a shorter simulation and then plotting solutions
num_samples = 10; α = 1; γ = 1;
u_3DVAR, u_TA3DVAR, error_3DVAR, error_TA3DVAR = run_simulation(num_samples, α, γ);

In [ ]:
contour(x,y,u_exact)
title!("Exact Solution, α=γ=1")
xlabel!("x")
ylabel!("y")
savefig("u_exact")

In [ ]:
contour(x,y,real(u_3DVAR))
title!("Classic 3DVAR Solution, 10 iterations, α=γ=1")
xlabel!("x")
ylabel!("y")
savefig("u_3DVAR")

In [ ]:
contour(x,y,real(u_TA3DVAR))
title!("Time Averaged 3DVAR Solution, 10 iterations, α=γ=1")
xlabel!("x")
ylabel!("y")
savefig("u_TA3DVAR")